<a href="https://colab.research.google.com/github/pastrop/kaggle/blob/master/Kaggle_Covid19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Use seaborn for the correlation heatmap
!pip install seaborn

In [0]:
import warnings
import numpy as np
import pandas as pd
#import seaborn as sns
import matplotlib.pyplot as plt
import itertools

# Time related libraries
import time

In [2]:
# file upload while using Google Colab
from google.colab import files
uploaded = files.upload()

Saving covid19-global-forecasting-week-1.zip to covid19-global-forecasting-week-1.zip


In [3]:
#Unzipping if needed
!unzip covid19-global-forecasting-week-1.zip

Archive:  covid19-global-forecasting-week-1.zip
  inflating: submission.csv          
  inflating: test.csv                
  inflating: train.csv               


In [16]:
#read file & check the upload
df = pd.read_csv('train.csv')
df_filtered = df.loc[df['ConfirmedCases'] != 0]
df_filtered.head(40)

,Id,Province/State,Country/Region,Lat,Long,Date,ConfirmedCases,Fatalities
33,34,NaN,Afghanistan,33.0000,65.0000,2020-02-24,1.0,0.0
34,35,NaN,Afghanistan,33.0000,65.0000,2020-02-25,1.0,0.0
35,36,NaN,Afghanistan,33.0000,65.0000,2020-02-26,1.0,0.0
36,37,NaN,Afghanistan,33.0000,65.0000,2020-02-27,1.0,0.0
37,38,NaN,Afghanistan,33.0000,65.0000,2020-02-28,1.0,0.0
38,39,NaN,Afghanistan,33.0000,65.0000,2020-02-29,1.0,0.0
39,40,NaN,Afghanistan,33.0000,65.0000,2020-03-01,1.0,0.0
40,41,NaN,Afghanistan,33.0000,65.0000,2020-03-02,1.0,0.0
41,42,NaN,Afghanistan,33.0000,65.0000,2020-03-03,1.0,0.0
42,43,NaN,Afghanistan,33.0000,65.0000,2020-03-04,1.0,0.0


In [6]:
data_shape = df.shape
print(data_shape)

(16188, 8)


In [7]:
#data cleaning & exploration
unknown_count = df.isna().sum().drop_duplicates()
unknown_count[unknown_count>0]

Province/State    8778
dtype: int64

In [0]:
df['Country/Region'].unique()

In [0]:
df_US = df[df['Country/Region']=='US']
df_US.head(30)

In [15]:
df_US_filtered = df_US.loc[df['ConfirmedCases'] != 0]
df_US_filtered.head(30)

,Id,Province/State,Country/Region,Lat,Long,Date,ConfirmedCases,Fatalities
12249,19954,Alabama,US,32.3182,-86.9023,2020-03-13,5.0,0.0
12250,19955,Alabama,US,32.3182,-86.9023,2020-03-14,6.0,0.0
12251,19956,Alabama,US,32.3182,-86.9023,2020-03-15,12.0,0.0
12252,19957,Alabama,US,32.3182,-86.9023,2020-03-16,29.0,0.0
12253,19958,Alabama,US,32.3182,-86.9023,2020-03-17,39.0,0.0
12254,19959,Alabama,US,32.3182,-86.9023,2020-03-18,46.0,0.0
12306,20047,Alaska,US,61.3707,-152.4044,2020-03-13,1.0,0.0
12307,20048,Alaska,US,61.3707,-152.4044,2020-03-14,1.0,0.0
12308,20049,Alaska,US,61.3707,-152.4044,2020-03-15,1.0,0.0
12309,20050,Alaska,US,61.3707,-152.4044,2020-03-16,1.0,0.0
